In [ ]:
# or simply
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-b0031612-4d1e-f20e-7735-3e306741e5e9)


In [ ]:
%%shell

pip install cython
# Install pycocotools, the version by default in Colab
# has a bug fixed in https://github.com/cocodataset/cocoapi/pull/354
pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-b_ki_cb9
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-b_ki_cb9
  Created wheel for pycocotools: filename=pycocotools-2.0-cp37-cp37m-linux_x86_64.whl size=263916 sha256=c1b3e759f40aa6d8551827506bd307ae834f31668fea767faf2957ae9e24b83d
  Stored in directory: /tmp/pip-ephem-wheel-cache-x__dzuj2/wheels/90/51/41/646daf401c3bc408ff10de34ec76587a9b3ebfac8d21ca5c3a
Successfully built pycocotools
  Found existing installation: pycocotools 2.0.2
    Uninstalling pycocotools-2.0.2:
      Successfully uninstalled pycocotools-2.0.2


In [ ]:
%%shell

# Download TorchVision repo to use some files from
# references/detection
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.3.0

cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
#cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

Cloning into 'vision'...
remote: Enumerating objects: 23788, done.
remote: Counting objects: 100% (1260/1260), done.
remote: Compressing objects: 100% (347/347), done.
remote: Total 23788 (delta 911), reused 1188 (delta 889), pack-reused 22528
Receiving objects: 100% (23788/23788), 30.84 MiB | 27.80 MiB/s, done.
Resolving deltas: 100% (17602/17602), done.
Note: checking out 'v0.3.0'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at be376084 version check against PyTorch's CUDA version


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !cp '/content/drive/MyDrive/Thesis/Pythorch Mask-RCNN/coco_utils.py' ./
!cp '/content/drive/MyDrive/Thesis/Pythorch Mask-RCNN/engine.py' ./

In [ ]:
# -*- coding: utf-8 -*-


"""## Initial imports"""

from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

"""### install pycocotools"""

# Commented out IPython magic to ensure Python compatibility.
# %%shell
# 
# pip install cython
# # Install pycocotools, the version by default in Colab
# # has a bug fixed in https://github.com/cocodataset/cocoapi/pull/354
# pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'




# from .vision import VisionDataset
from PIL import Image
import os
import os.path
from typing import Any, Callable, Optional, Tuple, List

from engine import train_one_epoch, evaluate
import utils

"""---

## Load dataset CityScapes in COCO format using a custom class
"""

from pycocotools import mask as maskUtils

"""### create a custom class to load properly the dataset CityScapes in coco format compatible with the train_one_epoch method"""

from torchvision.datasets.vision import VisionDataset
from pycocotools import mask as maskUtils

class my_CocoDetection(VisionDataset):
    """`MS Coco Detection <https://cocodataset.org/#detection-2016>`_ Dataset.

    Args:
        root (string): Root directory where images are downloaded to.
        annFile (string): Path to json annotation file.
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.ToTensor``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
        transforms (callable, optional): A function/transform that takes input sample and its target as entry
            and returns a transformed version.
    """

    def __init__(self, root, annFile, transforms = None):
        self.root = root
        self.annFile = annFile
        self.transforms = transforms

            # root: str,
            # annFile: str,
            # do_transforms=None
            # transform: Optional[Callable] = None,
            # target_transform: Optional[Callable] = None,
            # transforms: Optional[Callable] = None,
    # ) -> None:
    #     super(my_CocoDetection, self).__init__(root, annFile) #, transform, target_transform)
        from pycocotools.coco import COCO
        self.coco = COCO(annFile)
        self.ids = list(sorted(self.coco.imgs.keys()))

    def __getitem__(self, index: int) -> Tuple[Any, Any]:
        """
        Args:
            index (int): Index

        Returns:
            tuple: Tuple (image, target). target is the object returned by ``coco.loadAnns``.
        """
        coco = self.coco
        img_id = self.ids[index]
        ann_ids = coco.getAnnIds(imgIds=img_id)
        target = coco.loadAnns(ann_ids)

        path = coco.loadImgs(img_id)[0]['file_name']
        img = Image.open(os.path.join(self.root, path)).convert('RGB')

        # # resize the image to lower resolution
        # img = img.resize((1024, 512))

        width = img.size[0]
        heigth = img.size[1]

        boxes = []
        masks = []
        class_ids = []
        image_id = index
        iscrowd_list = []
        areas = []

        for elem in target:

          if(len(elem['bbox'])<4):
            print("!!!WARNING!! - found non 4 boxes for element")
            print("skipping to next image...")
            continue

          # sort the box coordinates in the right order
          xmin = elem['bbox'][0] 
          xmax = xmin + elem['bbox'][2 ]
          ymin = elem['bbox'][1] 
          ymax = ymin + elem['bbox'][3] 

          # assert xmin < xmax, "xmin should be less than xmax"
          # assert ymin < ymax, "ymin should be less than ymax"

          boxes.append([xmin, ymin, xmax, ymax])

          mask = self.annToMask(elem['segmentation'], heigth, width)
          # # uncomment tp resize mask to match image size
          # mask = Image.fromarray(mask) # convert np image in PIL
          # mask = mask.resize((1024, 512)) # resize
          # mask = np.array(mask) # convert back
          masks.append(mask)
          class_ids.append(elem['category_id'])
          #image_ids.append(elem['image_id'])
          iscrowd_list.append(elem['iscrowd'])
          # compute area and append to list
          areas.append(elem['area']) # np.int(elem['area']/4))

        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        masks = torch.as_tensor(masks, dtype=torch.uint8)
        class_ids = torch.as_tensor(class_ids, dtype=torch.int64)
        image_id = torch.tensor(image_id)
        iscrowd_list = torch.as_tensor(iscrowd_list, dtype=torch.int64)
        areas = torch.as_tensor(areas, dtype=torch.float32)

        # changed the name inside brackets because this format is required
        # in the train_one_epoch method
        gt_instances = {}
        gt_instances['boxes'] = boxes
        gt_instances['masks'] = masks
        gt_instances['labels'] = class_ids
        gt_instances['image_id'] = image_id
        gt_instances['iscrowd'] = iscrowd_list
        gt_instances['area'] = areas


        if self.transforms is not None:
            img, gt_instances = self.transforms(img, gt_instances)

        return img, gt_instances


    def __len__(self) -> int:
        return len(self.ids)


    def annToRLE(self, ann, height, width):
        """
        Convert annotation which can be polygons, uncompressed RLE to RLE.
        :return: binary mask (numpy 2D array)
        """
        segm = ann #ann['segmentation']
        if isinstance(segm, list):
            # polygon -- a single object might consist of multiple parts
            # we merge all parts into one mask rle code
            rles = maskUtils.frPyObjects(segm, height, width)
            rle = maskUtils.merge(rles)
        elif isinstance(segm['counts'], list):
            # uncompressed RLE
            rle = maskUtils.frPyObjects(segm, height, width)
        else:
            # rle
            rle = ann['segmentation']
        return rle

    def annToMask(self, ann, height, width):
        """
        Convert annotation which can be polygons, uncompressed RLE, or RLE to binary mask.
        :return: binary mask (numpy 2D array)
        """
        rle = self.annToRLE(ann, height, width)
        m = maskUtils.decode(rle)
        return m

cityscapes_dataset = my_CocoDetection(root='/content/drive/MyDrive/Thesis/datasets/cityscapes', 
                                  annFile='/content/drive/MyDrive/Thesis/datasets/cityscapes/annotations/instancesonly_filtered_gtFine_train.json' )

# print an instance to check data 
cityscapes_dataset[2]

print("Image Count: {}".format(len(cityscapes_dataset.ids)))

# in results I obtain a tuple where the first element if the PIL image 
# corresponding to that dataset index, and the second element is a dictionary
# containing all the field of the GT informations about the image, from 
# bbox to masks, class id etc.. 
results = cityscapes_dataset[2]
results[0].size

print(type(results[1]))
results[1].keys()

# plot the masks corresponding to the GT instances inside the image, 
# together with their classes

gt_mask = results[1]['masks']
gt_mask = gt_mask.mul(255).permute(1, 2, 0).byte().numpy()

gt_classes = results[1]['labels']


"""## Defining the model"""

import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

      
def get_instance_segmentation_model(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

"""

Let's write some helper functions for data augmentation / transformation, which leverages the functions in `refereces/detection` that we have just copied:
"""

from engine import train_one_epoch, evaluate
import utils
import transforms as T

def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    # if train:
    #     # during training, randomly flip the training images
    #     # and ground-truth for data augmentation
    #     transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

do_transform = get_transform(train=True)

"""Now we have the dataset and the model, let's prepare them for training"""

# use our dataset and defined transformations
dataset = my_CocoDetection('/content/drive/MyDrive/Thesis/datasets/cityscapes', 
                          '/content/drive/MyDrive/Thesis/datasets/cityscapes/annotations/instancesonly_filtered_gtFine_train.json', 
                          get_transform(train=True))

dataset_test = my_CocoDetection('/content/drive/MyDrive/Thesis/datasets/cityscapes', 
                              '/content/drive/MyDrive/Thesis/datasets/cityscapes/annotations/instancesonly_filtered_gtFine_train.json',
                              get_transform(train=True))

# split the dataset in train and test set
torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()

dataset_train = torch.utils.data.Subset(dataset, indices[:300])
dataset_test = torch.utils.data.Subset(dataset_test, indices[500:800])

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset_train, batch_size=1, shuffle=True, num_workers=2,
    collate_fn=utils.collate_fn)


data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=True, num_workers=2,
    collate_fn=utils.collate_fn)



"""instantiate the model and the optimizer"""

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has two classes only - background and person
num_classes = 9

# get the model using our helper function
model = get_instance_segmentation_model(num_classes)
# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

## Load the pretrained model
# PATH = "/content/drive/MyDrive/Thesis/Pythorch Mask-RCNN/saved_models/maskrcnn_resnet50_cityscapes_300imgs_epoch_"
PATH = "/content/drive/MyDrive/Thesis/Pythorch Mask-RCNN/saved_models_cityscapes/GAN_training_100%/generator/generator_300imgs_epoch_"

epoch = 12  # last epoch for which the model was saved
checkpoint = torch.load(PATH+str(epoch))
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

epoch = checkpoint['epoch']

model.train()

print("model loaded at epoch", str(epoch))


"""Before iterating over the dataset, it’s good to see what the model expects during training and inference time on sample data."""

# # for training
# images,targets = next(iter(data_loader))
# images = list(image for image in images)
# targets = [{k: v for k, v in t.items()} for t in targets]
# output = model(images,targets)   # Returns losses and detections
# # for inference
# model.eval()
# x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
# predictions = model(x)

"""We are ready to train the model"""

#PATH = "./saved_models/maskrcnn_resnet50_cityscapes_500imgs_epoch_"


# if Commented the model has already been trained and saved

# let's train it

# do just one further epoch and then proceed with GAN training
num_epochs = 1

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()

    # if (epoch%3 == 0):
    #   torch.save(model, PATH+str(epoch))

    # # evaluate on the test dataset
    # evaluate(model, data_loader_test, device=device)

# torch.save({
#             'epoch': epoch,
#             'model_state_dict': model.state_dict(),
#             'optimizer_state_dict': optimizer.state_dict(),
#             #'loss': loss,
#             }, PATH+str(epoch))

"""---

## Now that the model is saved, it can be loaded to do inference
"""

GPU = False

if torch.cuda.is_available():
  GPU = True

# --------- load with disctionary load:
"""
epoch = 1 # last epoch for ehich the model was saved

model = get_instance_segmentation_model(num_classes)
optimizer = optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

checkpoint = torch.load(PATH+str(epoch), map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
#loss = checkpoint['loss']

# model.eval()
# - or -
model.train()

print("model loaded")
"""



"""Printing the prediction shows that we have a list of dictionaries. Each element of the list corresponds to a different image. As we have a single image, there is a single dictionary in the list.
The dictionary contains the predictions for the image we passed. In this case, we can see that it contains `boxes`, `labels`, `masks` and `scores` as fields.
"""





def compute_matches(gt_boxes, gt_class_ids, gt_masks,
                  pred_boxes, pred_class_ids, pred_scores, pred_masks,
                  iou_threshold=0.5, score_threshold=0.0):
  
  """Finds matches between prediction and ground truth instances.
  Returns:
      gt_match: 1-D array. For each GT box it has the index of the matched
                predicted box.
      pred_match: 1-D array. For each predicted box, it has the index of
                  the matched ground truth box.
      overlaps: [pred_boxes, gt_boxes] IoU overlaps.
  """
  # Trim zero padding
  # TODO: cleaner to do zero unpadding upstream
  gt_boxes = trim_zeros(gt_boxes)
  gt_masks = gt_masks[..., :gt_boxes.shape[0]]
  pred_boxes = trim_zeros(pred_boxes)
  pred_scores = pred_scores[:pred_boxes.shape[0]]
  # Sort predictions by score from high to low
  indices = np.argsort(pred_scores)[::-1]
  pred_boxes = pred_boxes[indices]
  pred_class_ids = pred_class_ids[indices]
  pred_scores = pred_scores[indices]
  pred_masks = pred_masks[..., indices]

  # Compute IoU overlaps [pred_masks, gt_masks]
  overlaps = compute_overlaps_masks(pred_masks, gt_masks)

  # Loop through predictions and find matching ground truth boxes
  match_count = 0
  pred_match = -1 * np.ones([pred_boxes.shape[0]])
  gt_match = -1 * np.ones([gt_boxes.shape[0]])
  for i in range(len(pred_boxes)):
      # Find best matching ground truth box
      # 1. Sort matches by score
      sorted_ixs = np.argsort(overlaps[i])[::-1]
      # 2. Remove low scores
      low_score_idx = np.where(overlaps[i, sorted_ixs] < score_threshold)[0]
      if low_score_idx.size > 0:
          sorted_ixs = sorted_ixs[:low_score_idx[0]]
      # 3. Find the match
      for j in sorted_ixs:
          # If ground truth box is already matched, go to next one
          if gt_match[j] > -1:
              continue
          # If we reach IoU smaller than the threshold, end the loop
          iou = overlaps[i, j]
          if iou < iou_threshold:
              break
          # Do we have a match?
          if pred_class_ids[i] == gt_class_ids[j]:
              match_count += 1
              gt_match[j] = i
              pred_match[i] = j
              break

  return gt_match, pred_match, overlaps

def trim_zeros(x):
  """It's common to have tensors larger than the available data and
  pad with zeros. This function removes rows that are all zeros.
  x: [rows, columns].
  """
  assert len(x.shape) == 2
  return x[~np.all(x == 0, axis=1)]


def compute_overlaps_masks(masks1, masks2):
    """Computes IoU overlaps between two sets of masks.
    masks1, masks2: [Height, Width, instances]
    """
    
    # If either set of masks is empty return empty result
    if masks1.shape[-1] == 0 or masks2.shape[-1] == 0:
        return np.zeros((masks1.shape[-1], masks2.shape[-1]))
    # flatten masks and compute their areas
    masks1 = np.reshape(masks1 > .5, (-1, masks1.shape[-1])).astype(np.float32)
    masks2 = np.reshape(masks2 > .5, (-1, masks2.shape[-1])).astype(np.float32)
    area1 = np.sum(masks1, axis=0)
    area2 = np.sum(masks2, axis=0)

    # intersections and union
    intersections = np.dot(masks1.T, masks2)
    union = area1[:, None] + area2[None, :] - intersections
    overlaps = intersections / union

    return overlaps



loading annotations into memory...
Done (t=5.81s)
creating index...
index created!
Image Count: 2975
<class 'dict'>
loading annotations into memory...
Done (t=3.85s)
creating index...
index created!
loading annotations into memory...
Done (t=5.84s)
creating index...
index created!
model loaded at epoch 12
Epoch: [0]  [  0/300]  eta: 0:31:32  lr: 0.000001  loss: 1.1853 (1.1853)  loss_classifier: 0.2336 (0.2336)  loss_box_reg: 0.2634 (0.2634)  loss_mask: 0.2362 (0.2362)  loss_objectness: 0.0527 (0.0527)  loss_rpn_box_reg: 0.3994 (0.3994)  time: 6.3085  data: 5.4137  max mem: 1827
Epoch: [0]  [ 10/300]  eta: 0:10:50  lr: 0.000008  loss: 0.8402 (0.7597)  loss_classifier: 0.1183 (0.1383)  loss_box_reg: 0.2634 (0.2525)  loss_mask: 0.2362 (0.2395)  loss_objectness: 0.0146 (0.0168)  loss_rpn_box_reg: 0.0470 (0.1126)  time: 2.2432  data: 1.8179  max mem: 2096
Epoch: [0]  [ 20/300]  eta: 0:15:10  lr: 0.000014  loss: 0.8553 (0.8938)  loss_classifier: 0.1424 (0.1674)  loss_box_reg: 0.2952 (0.3057)

In [ ]:
"""---

# GAN part
### Now that we have our Mask-RCNN Model that is ready and trained on the dataset, we try to improve its prediction performances by adversarial approach.

#### Creation of discriminator network
"""

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import torchvision.utils as vutils

from math import sqrt
import matplotlib.pyplot as plt
import numpy as np
###################################
## -------- Configs ------------ ##
###################################

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

# Number of channels in the training images. For color images this is 3
nc = 1

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Number of training epochs for discriminators train
num_epochs = 10

# Number of training epochs for GAN training
num_epochs_GAN = 2

"""### Mask Discriminator network:"""

class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            
            # input is (nc) x 128 x 128
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            # input is (nc) x 64 x 64
            nn.Conv2d(ndf, ndf*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf*2, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4

            nn.Conv2d(ndf * 8, ndf * 16, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 16),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*16) x 2 x 2

            nn.Conv2d(ndf * 16, 1, 2, 1, 0, bias=False),
            # nn.Conv2d(ndf * 16, 1, 2, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

"""### Boxes discriminator network:"""

class BoxDiscriminator(nn.Module):
    def __init__(self, nc=256, ndf=64): # channel dim is 256 because it takes as input the feature map
        super(BoxDiscriminator, self).__init__()
        self.main = nn.Sequential(
            # input is (channel_dim) x 32 x 32
            nn.Conv2d(256, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 16 x 16
            nn.Conv2d(ndf * 1, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 8 x 8
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 4 x 4
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 2 x 2
            nn.Conv2d(ndf * 8, 1, 2, 1, 0, bias=False),
            nn.Sigmoid()
        )


    def forward(self, input):
      return self.main(input)

"""### Weight init

From the DCGAN paper, the authors specify that all model weights shall be randomly initialized from a Normal distribution with mean=0, stdev=0.02. The weights_init function takes an initialized model as input and reinitializes all convolutional, convolutional-transpose, and batch normalization layers to meet this criteria. This function is applied to the models immediately after initialization.
"""

# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

# Create the Mask Discriminator
netD = Discriminator(ngpu).to(device)

# Create the Box Discriminator
netD_box = BoxDiscriminator(ngpu).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netD = nn.DataParallel(netD, list(range(ngpu)))
    netD_box = nn.DataParallel(netD, list(range(ngpu)))

# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
netD.apply(weights_init)
netD_box.apply(weights_init)

# Print the model
# print(netD)
# print(netD_box)

from torchsummary import summary

summary(netD, (1, 128, 128))
# summary(netD_box, (256, 32, 32))

"""### Loss function and optimizer"""

# Initialize BCELoss function
criterion = nn.BCELoss()

#Initialize loss function for the Mask generator training
mae = nn.L1Loss() # MAE loss

# Establish convention for real and fake labels during training
real_label = 1.
fake_label = 0.

# Optimizer for the discriminators:
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerD_box = optim.Adam(netD_box.parameters(), lr=lr, betas=(beta1, 0.999))

# optimizer for the generator
optimizer = torch.optim.SGD(params, lr=lr,
                            momentum=0.9, weight_decay=0.00002)
# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)



----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]           1,024
         LeakyReLU-2           [-1, 64, 64, 64]               0
            Conv2d-3          [-1, 128, 32, 32]         131,072
       BatchNorm2d-4          [-1, 128, 32, 32]             256
         LeakyReLU-5          [-1, 128, 32, 32]               0
            Conv2d-6          [-1, 128, 16, 16]         262,144
       BatchNorm2d-7          [-1, 128, 16, 16]             256
         LeakyReLU-8          [-1, 128, 16, 16]               0
            Conv2d-9            [-1, 256, 8, 8]         524,288
      BatchNorm2d-10            [-1, 256, 8, 8]             512
        LeakyReLU-11            [-1, 256, 8, 8]               0
           Conv2d-12            [-1, 512, 4, 4]       2,097,152
      BatchNorm2d-13            [-1, 512, 4, 4]           1,024
        LeakyReLU-14            [-1, 51

In [ ]:
### load the discriminator
#load discriminator
PATH_D = "/content/drive/MyDrive/Thesis/Pythorch Mask-RCNN/saved_models_cityscapes/GAN_training_100%/discriminators/mask_discriminator_300imgs_epoch_"

epoch = 12  # last epoch for which the model was saved
checkpoint = torch.load(PATH_D+str(epoch))
netD.load_state_dict(checkpoint['model_state_dict'])
optimizerD.load_state_dict(checkpoint['optimizer_state_dict'])

epoch = checkpoint['epoch']
netD.train()

Discriminator(
  (main): Sequential(
    (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (12): BatchNorm2d(512, eps=1e-05, 

## GAN training

In [ ]:

import torch.nn.functional as nnf
import torchvision
import torchvision.transforms as transforms

tensor_converter = transforms.ToTensor()



"""---

## Adversarial training

### Now it's time to put the Mask-RCNN generator in competition with the Masks discriminator and the boxes discriminatir networks

## Adversarial training with GT - Predictions match
"""

SAVE_PATH = "/content/drive/MyDrive/Thesis/Pythorch Mask-RCNN/saved_models_cityscapes/"
last_epoch = 9


# Training Loop

# Lists to keep track of progress
img_list = []

iters = 0
masks = []
b_size = 1

LAMBDA = 100

start_epoch = 13

num_epochs = 20

# errD_masks = torch.zeros((1), device=device, dtype=dtype)
# errG_tot = torch.zeros((1), device=device, dtype=dtype)

print("Starting Training Loop...")
# For each epoch
for epoch in range(start_epoch, num_epochs + start_epoch):

    D_mask_losses = []
    Gen_losses = []
    # For each batch in the dataloader
    for k, data in enumerate(data_loader, 0):

        if (len(data[1][0]["boxes"]) == 0):
            continue

        # get the image
        tensor_image = data[0][0]
        # get the corresponding gt boxes
        gt_boxes = data[1][0]["boxes"].cpu().numpy()
        # get the corresponding gt masks
        gt_masks = data[1][0]["masks"]
        tensor_gt_labels = data[1][0]["labels"]
        ## -----------------------
        # generate mask predictions using the Mask-RCNN model
        model.eval()  # put the model in evaluation mode

        prediction = model([tensor_image.to(device)])

        ## IMPORTANT restore the model in training mode
        model.train()
        # -------
        pred_masks = prediction[0]["masks"]  # tensor predicted masks
        # save predictions and extract each corresponding boxes
        pred_boxes = prediction[0]["boxes"].cpu().detach().numpy()
        # save predicted scores and labels
        pred_labels = prediction[0]["labels"].cpu().numpy()
        pred_scores = prediction[0]["scores"].cpu().detach().numpy()

        # ################ Compute Matches ######################
        ##
        # get the corresponding gt numpy masks
        gt_numpy_masks = data[1][0]["masks"].mul(255).permute(1, 2, 0).cpu().numpy()
        ## -----------------------
        pred_numpy_masks = torch.squeeze(prediction[0]["masks"], dim=1).permute(1, 2, 0).cpu().detach().numpy()
        ## --------------------------------
        gt_match, pred_match, overlaps = compute_matches(gt_boxes, tensor_gt_labels, gt_numpy_masks,
                                                         pred_boxes, pred_labels, pred_scores, pred_numpy_masks)
        valid_gt_match = []  # initialize list before every iteration to avoid errors
        valid_gt_elems = []
        index = 0  # initialize i before every iteration to avoid errors
        elem = None  # initialize i before every iteration to avoid errors
        for index, elem in enumerate(gt_match):
            if np.int(elem) != -1:
                valid_gt_match.append(np.int(elem))
                valid_gt_elems.append(index)

        # ################ END - Compute Matches ######################

        # inizialize gradients at each iteration
        netD.zero_grad()

        # # uncomment to block discriminator training in even epochs
        # netD.train()
        # if (k%2 == 0):
        #   netD.train(False)

        real_errors = []
        fake_errors = []
        gen_losses = []

        errD_masks = 0
        errG_tot = 0

        real_batch = torch.empty(gt_masks.shape[0], 1, 128, 128)
        fake_batch = torch.empty(pred_masks.shape[0], 1, 128, 128)

        i = 0  # initialize i before every iteration to avoid errors
        for i in range(gt_masks.shape[0]):
            real_mask = gt_masks[i, :, :]  # get the i-th mask
            # add channel
            real_mask.unsqueeze_(0)
            normalized_real_mask = (real_mask - 0.5).div(0.5)  # normalize the mask in -1,1
            # add batch dimension
            # normalized_real_mask.unsqueeze_(0)

            real_box = gt_boxes[i, :]
            top = int(real_box[1])
            left = int(real_box[0])
            heigth = int(real_box[3] - real_box[1])
            width = int(real_box[2] - real_box[0])

            ## REAL DATA
            # crop real mask with real boxes
            cropped_mask = torchvision.transforms.functional.crop(normalized_real_mask, top, left, heigth, width)
            # resize the cropped mask to 128x128
            tensor_mask = transforms.functional.resize(cropped_mask, [128, 128])

            real_batch[i] = tensor_mask

        # train with reals
        if (np.random.uniform() > 0.95):  # introduce a low probability of noisy labels)
            label = torch.FloatTensor(gt_masks.shape[0], ).uniform_(0.0, 0.3)
            # label = torch.full((gt_masks.shape[0],), fake_label, dtype=torch.float, device=device)
        else:
            label = torch.FloatTensor(gt_masks.shape[0], ).uniform_(0.7, 1.2)
            # label = torch.full((gt_masks.shape[0],), real_label, dtype=torch.float, device=device)

        if (GPU == True):
            label, real_batch = label.cuda(), real_batch.cuda()
        output = netD(real_batch).view(-1)
        # Calculate loss on all-real batch

        errD_real_mask = criterion(output, label)
        # print("error real masks ",errD_real_mask)
        # print("error real requires grad ",errD_real_mask.requires_grad)

        errD_real_mask.backward()

        D_x_mask_real = output.mean().item()

        ##  ----- TRAIN WITH FAKES -----------

        j = 0
        for j in range(pred_masks.shape[0]):
            fake_mask = pred_masks[j, :, :, :]
            normalized_fake_mask = (fake_mask - 0.5).div(0.5)  # normalize the mask in -1,1
            # print("normalized fake mask shape ", normalized_fake_mask.shape )

            fake_box = pred_boxes[j, :]
            top = int(fake_box[1])
            left = int(fake_box[0])
            heigth = int(fake_box[3] - fake_box[1])
            width = int(fake_box[2] - fake_box[0])

            # crop using the predicted boxes
            cropped_fake_mask = torchvision.transforms.functional.crop(normalized_fake_mask, top, left, heigth, width)
            # print("cropped fake mask shape ", cropped_fake_mask.shape )
            # resize the cropped mask to 128x128
            tensor_fake_mask = transforms.functional.resize(cropped_fake_mask, [128, 128])
            # print("tensor fake mask shape ", tensor_fake_mask.shape)
            fake_batch[j] = tensor_fake_mask

        real_matched_batch = torch.empty(len(valid_gt_match), 1, 128, 128)
        fake_matched_batch = torch.empty(len(valid_gt_match), 1, 128, 128)
        count = 0

        h = 0
        for h in range(pred_masks.shape[0]):

            matching = False
            ### Chech if there is matching with GT ###
            if (h in valid_gt_match):  # check if there is matching
                matching = True
                # we have a prediction - gt correspondence
                # get index elem of h in valid gt elems
                corresponding_index = valid_gt_match.index(h)
                # use the index to find valid gt index in gt masks
                valid_gt_index = valid_gt_elems[corresponding_index]
                # get the mask
                real_mask = real_batch[valid_gt_index]  # get the i-th real mask
                fake_mask = fake_batch[h]

                real_matched_batch[count] = real_mask
                fake_matched_batch[count] = fake_mask

                count += 1

        # train with fakes
        if (np.random.uniform() > 0.95):  # introduce a low probability of noisy labels)
            # label = torch.full((pred_masks.shape[0],), real_label, dtype=torch.float, device=device)
            label = torch.FloatTensor(pred_masks.shape[0], ).uniform_(0.7, 1.2)
        else:
            label = torch.FloatTensor(pred_masks.shape[0], ).uniform_(0.0, 0.3)
            # label = torch.full((pred_masks.shape[0],), fake_label, dtype=torch.float, device=device)

        if (GPU == True):
            label, fake_batch = label.cuda(), fake_batch.cuda()
        output = netD(fake_batch.detach()).view(-1)
        # Calculate loss on all-fake batch

        errD_fake_mask = criterion(output, label)
        # print("error fake masks ",errD_fake_mask)
        # print("error fake requires grad ",errD_fake_mask.requires_grad)
        errD_fake_mask.backward()

        D_x_mask_fake = output.mean().item()

        errD_masks = float(errD_real_mask + errD_fake_mask)

        # Update D
        optimizerD.step()

        # free some GPU space
        # del label, errD_fake_mask, errD_real_mask, tensor_image, gt_masks, real_batch
        # torch.cuda.empty_cache()

        # --------------------
        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        model.zero_grad()


        # set label to true label
        label = torch.full((pred_masks.shape[0],), real_label, dtype=torch.float, device=device)
        if (GPU == True):
            real_matched_batch, fake_matched_batch, label = real_matched_batch.cuda(), fake_matched_batch.cuda(), label.cuda()

        # Since we just updated D, perform another forward pass of all-fake batch through D
        # compute output for Mask
        outputM = netD(fake_batch).view(-1)

        # Calculate G's loss based on this output
        if (matching == True):
            errG_mask = criterion(outputM, label) + LAMBDA * mae(real_matched_batch, fake_matched_batch)
        else:
            errG_mask = criterion(outputM, label)

        # Calculate gradients for G
        errG_mask.backward()
        # plot_grad_flow(model.named_parameters()) # plots the gradient flow
        optimizer.step()
        lr_scheduler.step()

        D_G_mask = outputM.mean().item()

        errG_tot = float(errG_mask)

        # Output training stats
        if k % 5 == 0:
            print('[%d/%d][%d/%d]\tLoss_D_mask: %.4f\tLoss_G: %.4f\tD(x_real): %.4f\tD(x_fake): %.4f\t'
                  % (epoch, num_epochs, k, len(data_loader),
                     errD_masks, errG_tot, D_x_mask_real, D_x_mask_fake))

        D_mask_losses.append(errD_masks)
        Gen_losses.append(errG_tot)


    if(epoch % 4 == 0):
        # save models
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            # 'loss': loss,
        }, SAVE_PATH+"GAN_training_100%/generator/generator_300imgs_epoch_" + str(epoch))

        torch.save({
            'epoch': epoch,
            'model_state_dict': netD.state_dict(),
            'optimizer_state_dict': optimizerD.state_dict(),
            # 'loss': loss,
        }, SAVE_PATH+"GAN_training_100%/discriminators/mask_discriminator_300imgs_epoch_" + str(epoch))

        print("models saved correctly")

        # save loss lists into file

    # D box losses
    f=open(SAVE_PATH+'GAN_training_100%/losses/D_mask_300_losses_epoch'+str(epoch)+'.txt','w')
    for ele in D_mask_losses:
        f.write(str(ele)+'\n')
    f.close()

    # generator losses
    f=open(SAVE_PATH+'GAN_training_100%/losses/generator_300_losses_epoch'+str(epoch)+'.txt','w')
    for ele in Gen_losses:
        f.write(str(ele)+'\n')
    f.close()


Starting Training Loop...
[13/20][0/300]	Loss_D_mask: 1.0873	Loss_G: 1.7119	D(x_real): 0.9514	D(x_fake): 0.2050	
[13/20][5/300]	Loss_D_mask: 1.0398	Loss_G: 2.1445	D(x_real): 0.9481	D(x_fake): 0.1239	
[13/20][10/300]	Loss_D_mask: 0.5097	Loss_G: 2.1199	D(x_real): 0.8610	D(x_fake): 0.1028	
[13/20][15/300]	Loss_D_mask: 0.6724	Loss_G: 1.8993	D(x_real): 0.9250	D(x_fake): 0.1458	
[13/20][20/300]	Loss_D_mask: 0.6939	Loss_G: 1.9888	D(x_real): 0.9187	D(x_fake): 0.1380	
[13/20][25/300]	Loss_D_mask: 0.8182	Loss_G: 1.9623	D(x_real): 0.9446	D(x_fake): 0.1471	
[13/20][30/300]	Loss_D_mask: 0.6950	Loss_G: 1.2687	D(x_real): 0.9493	D(x_fake): 0.2594	
[13/20][35/300]	Loss_D_mask: 0.6221	Loss_G: 1.1226	D(x_real): 0.9671	D(x_fake): 0.3191	
[13/20][40/300]	Loss_D_mask: 0.5995	Loss_G: 1.7554	D(x_real): 0.9462	D(x_fake): 0.1898	
[13/20][45/300]	Loss_D_mask: 0.8060	Loss_G: 1.7169	D(x_real): 0.9644	D(x_fake): 0.1898	
[13/20][50/300]	Loss_D_mask: 0.6146	Loss_G: 1.3925	D(x_real): 0.8758	D(x_fake): 0.2140	
[13/20][